## **Libraries**

In [ ]:
# Import libraries
import numpy as np
import os
import pandas as pd

## **Data**

In [ ]:
# Load raw data and make a copy
loan_data_backup = pd.read_csv(os.path.join("..", "data", "raw", "loan_data_0714.csv"))
loan_data = loan_data_backup.copy()
loan_data.head()

In [ ]:
# Display all column names
loan_data.columns.values

In [ ]:
# Display column names along with cases and datatype per column
loan_data.info()

### **Continuous Variables**

In [ ]:
# Display unique values of a column
loan_data["emp_length"].unique()

In [ ]:
# Process the variables related to employment length and store it in a new variable
loan_data["emp_lng_int"] = loan_data["emp_length"].str.replace("+ years", "")
loan_data["emp_lng_int"] = loan_data["emp_lng_int"].str.replace("< 1 year", str(0))
loan_data["emp_lng_int"] = loan_data["emp_lng_int"].str.replace("n/a", str(0))
loan_data["emp_lng_int"] = loan_data["emp_lng_int"].str.replace(" years", "")
loan_data["emp_lng_int"] = loan_data["emp_lng_int"].str.replace(" year", "")

In [ ]:
# Check the datatype of a single element of a column
type(loan_data["emp_lng_int"][0])

In [ ]:
# Transform the values to numeric
loan_data["emp_lng_int"] = pd.to_numeric(loan_data["emp_lng_int"])
type(loan_data["emp_lng_int"][0])

In [ ]:
# Display a specific column
loan_data["earliest_cr_line"]

In [ ]:
# Extract the date and the time from a string variable
loan_data["cr_line_date"] = pd.to_datetime(loan_data["earliest_cr_line"], format = "%b-%y")
type(loan_data["cr_line_date"][0])

In [ ]:
# Create a new column to compute the number of months since the earliest credit line date
loan_data["months_cr_line"] = round(pd.to_numeric((pd.to_datetime("2017-12-01") - loan_data["cr_line_date"]) / np.timedelta64(1, "M")))
loan_data["months_cr_line"].describe()

In [ ]:
# Set the rows that had negative differences to the maximum value
loan_data["months_cr_line"][loan_data["months_cr_line"] < 0] = loan_data["months_cr_line"].max()
min(loan_data["months_cr_line"])

In [ ]:
# Show some descriptive statistics for the values of a column
loan_data["term"].describe()

In [ ]:
# Check the datatype of a single element of a column
type(loan_data["term"][25])

In [ ]:
# Replace a string with an empty string and turn the result to numeric
loan_data["term_int"] = pd.to_numeric(loan_data["term"].str.replace(" months", ""))
loan_data["term_int"]

In [ ]:
# Extract the date and the time from a string variable that is in a given format
loan_data["issue_d_date"] = pd.to_datetime(loan_data["issue_d"], format = "%b-%y")

In [ ]:
# Calculate the difference between two dates in months and turn it to numeric datatype
loan_data["mths_issue_d"] = round(pd.to_numeric((pd.to_datetime("2017-12-01") - loan_data["issue_d_date"]) / np.timedelta64(1, "M")))

In [ ]:
# Show some descriptive statistics for the values of a column
loan_data["mths_issue_d"].describe()

### **Discrete Variables**

In [ ]:
# Create dummy variables from all eight original independent variables and save them into a list
loan_dummies  =  [pd.get_dummies(loan_data["grade"], prefix = "grade", prefix_sep = ":"),
                pd.get_dummies(loan_data["sub_grade"], prefix = "sub_grade", prefix_sep = ":"),
                pd.get_dummies(loan_data["home_ownership"], prefix = "home_ownership", prefix_sep = ":"),
                pd.get_dummies(loan_data["verification_status"], prefix = "verification_status", prefix_sep = ":"),
                pd.get_dummies(loan_data["loan_status"], prefix = "loan_status", prefix_sep = ":"),
                pd.get_dummies(loan_data["purpose"], prefix = "purpose", prefix_sep = ":"),
                pd.get_dummies(loan_data["addr_state"], prefix = "addr_state", prefix_sep = ":"),
                pd.get_dummies(loan_data["initial_list_status"], prefix = "initial_list_status", prefix_sep = ":")]

In [ ]:
# Concatenate the dummy variables to turn them into a data frame
loan_dummies = pd.concat(loan_dummies, axis = 1)
type(loan_data)

In [ ]:
# Concatenate the data frame with original data with the data frame with dummy variables
loan_data = pd.concat([loan_data, loan_dummies], axis = 1)
loan_data.columns.values

### **Missing Values**

In [ ]:
# Check the number of missing values in the dataframe
pd.options.display.max_rows = None
loan_data.isnull().sum()

In [ ]:
# Fill the missing values in the variable related to the total revolving high credit limit
loan_data["total_rev_hi_lim"].fillna(loan_data["funded_amnt"], inplace = True)
loan_data["total_rev_hi_lim"].isnull().sum()

In [ ]:
# Fill the missing values with the mean value of the non-missing values
loan_data["annual_inc"].fillna(loan_data["annual_inc"].mean(), inplace = True)
loan_data["annual_inc"].isnull().sum()

In [ ]:
# Fill the missing values with zeroes
loan_data["months_cr_line"].fillna(0, inplace = True)
loan_data["acc_now_delinq"].fillna(0, inplace = True)
loan_data["total_acc"].fillna(0, inplace = True)
loan_data["pub_rec"].fillna(0, inplace = True)
loan_data["open_acc"].fillna(0, inplace = True)
loan_data["inq_last_6mths"].fillna(0, inplace = True)
loan_data["delinq_2yrs"].fillna(0, inplace = True)
loan_data["emp_lng_int"].fillna(0, inplace = True)

### **Saving**

In [ ]:
# Save processed data for modelling
loan_data.to_csv(os.path.join("..", "data", "processed", "loan_data_0714.csv"), index = False)